In [2]:
import pandas as pd 
import random

In [ ]:
df_old = pd.read_csv("speeches_old_emi.csv")
df_new = pd.read_csv("speeches_new_emi.csv")

In [ ]:
def sample(df):
    sampled_df = pd.DataFrame()
    decades = df['decade'].unique()
    bins = df['evidence_intuition_bin'].unique()
    bins_count = 4  # We have created 4 bins

    for decade in decades:
        decade_df = df[df['decade'] == decade]

        # Calculate how many rows to sample from each bin
        per_bin_sample = 52 // bins_count

        for binn in bins:
            bin_df = decade_df[decade_df['evidence_intuition_bin'] == binn]
            if len(bin_df) >= per_bin_sample:
                sampled_bin = bin_df.sample(per_bin_sample, random_state=1)
            else:
                sampled_bin = bin_df

            sampled_df = pd.concat([sampled_df, sampled_bin])
            
    return sampled_df

In [ ]:
df_new
# Define the bins based on the min and max of the evidence_minus_intuition_score
min_val = df_new['evidence_minus_intuition_score'].min() - 0.00001
max_val = df_new['evidence_minus_intuition_score'].max() + 0.00001
bins = pd.interval_range(start=min_val, end=max_val, periods=4)

# Create the bins and add them to the dataframe
df_new['evidence_intuition_bin'] = pd.cut(df_new['evidence_minus_intuition_score'], bins=bins, labels=range(4))


# Converting the 'Date' column to datetime format
df_new['year'] = pd.to_datetime(df['date'])

# Extracting the year and replacing the 'Date' column with the year only
df_new['year'] = df['year'].dt.year

# Create a new column for decades
df_new['decade'] = (df_new['year'] // 10) * 10


# Apply the function to the DataFrame with party consideration
#sampled_df = sample_uniformly(df_new, 'decade', 'evidence_intuition_bin', 'factionId', sample_size=50, party=True)
sampled_df1 = sample(df_new)
sampled_df1 = sampled_df1[["id",'year', 'speechContent','evidence_minus_intuition_score']]
sampled_df1

In [ ]:
# Create a new column for decades

# Define the bins based on the min and max of the evidence_minus_intuition_score
min_val = df_old['evidence_minus_intuition_score'].min() - 0.00001
max_val = df_old['evidence_minus_intuition_score'].max() + 0.00001
bins = pd.interval_range(start=min_val, end=max_val, periods=4)

# Create the bins and add them to the dataframe
df_old['evidence_intuition_bin'] = pd.cut(df_old['evidence_minus_intuition_score'], bins=bins, labels=range(4))
df_old['decade'] = (df_old['dateYear'] // 10) * 10

sampled_df2 = sample(df_old)
sampled_df2 = sampled_df2[["id",'dateYear', 'speechContent', 'evidence_minus_intuition_score']]
try:
    sampled_df2 = sampled_df2.rename(columns={'dateYear': 'year'})
except:
    pass
sampled_df2

In [ ]:
sample = pd.concat([sampled_df1, sampled_df2], ignore_index=True)
sample.to_csv("sample.csv", index = False)